In [ ]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim

import os
import model as OCRModel

import matplotlib.pyplot as plt

In [ ]:
cwd=os.getcwd()
traindir=os.path.join(cwd, 'Preprocessed-Data','training-data')
testdir=os.path.join(cwd, 'Preprocessed-Data','testing-data')

In [ ]:
transform=transforms.Compose([transforms.ToTensor(),
                              transforms.Normalize(mean=[0.5894, 0.5894, 0.5894], std=[0.2729, 0.2729, 0.2729]),
                              transforms.Grayscale(num_output_channels=1),
                              transforms.RandomRotation(10),
                              transforms.RandomHorizontalFlip(),
                              transforms.RandomAffine(0, translate=(0.1, 0.1))])

In [ ]:
trainData=datasets.ImageFolder(traindir, transform=transform)
testData=datasets.ImageFolder(testdir, transform=transform)

In [ ]:
# parameters for dataloaders
batch_size=4
num_workers=2

trainloader=DataLoader(trainData,
                       batch_size=batch_size,
                       num_workers=num_workers)

testloader=DataLoader(testData,
                      batch_size=batch_size,
                      num_workers=num_workers)

In [ ]:
class_names=trainData.classes

In [ ]:
for images, labels in trainloader:
    print(images.shape, labels.shape)
    break

*Printing Data for Visualization*

In [ ]:
data_iter = iter(trainloader)
images, labels = next(data_iter)

# Print the shape of images and labels
print(images.shape, labels.shape)

# Function to unnormalize and show an image
def imshow(img):
    img = img * 0.5 + 0.5  # Unnormalize
    npimg = img.numpy()
    plt.imshow(npimg, cmap='gray')
    plt.show()

# Print one image from the batch
imshow(images[0][0])


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
model=OCRModel.myOCRModel(len(class_names)).to(device)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 5
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in trainloader:
        # Move data to the device (GPU)
        images, labels = images.to(device), labels.to(device)
        
        # Zero the parameter gradients
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward pass and optimization
        loss.backward()
        optimizer.step()
        
        # Accumulate loss
        running_loss += loss.item()
    
    print(f'Epoch {epoch+1}, Loss: {running_loss/len(trainloader)}')



In [ ]:
# Evaluation loop
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in testloader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy: {100 * correct / total}%')

In [ ]:
#SAVE YOUR MODEL
#torch.save(model.state_dict(), 'ocr_model.pth')